# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

4. Выполнить 3 с помощью "протягиваемых" формул.

## Лабораторная работа 7.1

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [35]:
import pandas as pd
import xlwings as xw
import numpy as np

reviews = pd.read_csv('reviews_sample.csv')
recipes = pd.read_csv('recipes_sample.csv')

recipes = recipes.drop(['contributor_id', 'n_steps'], axis=1)
recipes

,name,id,minutes,submitted,description,n_ingredients
0,george s at the cove black bean soup,44123,90,2002-10-25,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,2003-07-26,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,2002-08-29,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,2002-07-27,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,2004-02-23,i think a fondue is a very romantic casual din...,NaN
...,...,...,...,...,...,...
29995,zurie s holey rustic olive and cheddar bread,267661,80,2007-11-25,this is based on a french recipe but i changed...,10.0
29996,zwetschgenkuchen bavarian plum cake,386977,240,2009-08-24,"this is a traditional fresh plum cake, thought...",11.0
29997,zwiebelkuchen southwest german onion cake,103312,75,2004-11-03,this is a traditional late summer early fall s...,NaN
29998,zydeco soup,486161,60,2012-08-29,this is a delicious soup that i originally fou...,NaN


2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [62]:
# print(df.loc['b'])
# print(df.iloc[1])

rev_data = reviews.iloc[np.random.randint(0, reviews.shape[0], int(reviews.shape[0] * 0.05))] #срезы
rec_data = recipes.iloc[np.random.randint(0, recipes.shape[0], int(recipes.shape[0] * 0.05))]

xlrec = xw.Book('recipes.xlsx')
sheet_rec = xlrec.sheets.add(name="Рецепты", after=wb.sheets[0].name)
sheet_rev = xlrec.sheets.add(name="Отзывы", after=wb.sheets[0].name) #создание листов


In [63]:
sheet_rec.range('A1').value = rec_data #добавление датафреймов на листы
sheet_rev.range('A1').value = rev_data 

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [102]:
sheet_rec.range('H1').value = 'seconds_assign'
minutes = sheet_rec.range('D2').options(np.array, expand='down').value #собрали массив
sheet_rec.range('H2').options(transpose=True).value = minutes*60 #добавили в пустую ячейку

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [109]:
sheet_rec.range('I1').value = 'seconds_formula' #называем столбец

fml = sheet_rec.range('I2').formula = f'=D2 * 60' # пишем формулу
sheet_rec.range("I2").expand('down').formula = fml # протягиваем формулу

5. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [125]:
sheet_rec.range('H1', 'I1').options(expand='right').autofit() #автоматически подгоняем ширину ячейки
sheet_rec.range('H1', 'I1').font.bold = True #делаем полужирным
sheet_rec.range('H:H').api.HorizontalAlignment = -4108 #выравниваем по центру (-4108 соответствует центрированию)
sheet_rec.range('I:I').api.HorizontalAlignment = -4108

6. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [141]:
for i, j in enumerate(sheet_rec.range('D2').expand('down').options(ndim=1).value): #цикл по списку минут
    
    if j < 5:
        sheet_rec.range('D' + str(i + 2)).color = (0, 255, 0) #красим клетку в нужный цвет

    if 5 <= j <= 10:
        sheet_rec.range('D' + str(i + 2)).color = (255, 255, 0)

    if j > 5:
        sheet_rec.range('D' + str(i + 2)).color = (255, 0, 0)

7. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [163]:
sheet_rec.range('J1').value = 'n_reviews' #называем столбец

recps = sheet_rec.range('C2').expand('down').options(ndim=1).value
rng_name = sheet_rev.range('D2').expand('down').address

c=1
for i in recps:
    c+=1
    sheet_rec.range('J2' + str(c)).value = sheet_rev.formula = f'=COUNTIF(C:C, {i})'
    

## Лабораторная работа 7.2

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

9. В файле `recipes_model.csv` находится модель данных предметной области "рецепты". При помощи пакета `csv` считайте эти данные. При помощи пакета `xlwings` запишите данные на лист `Модель` книги `recipes_model.xlsx`, начиная с ячейки `A2`, не используя циклы. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука. 

10. При помощи пакета `xlwings` добавьте в столбец J формулу для описания столбца на языке SQL. Формула должна реализовывать следующую логику:

    1\. в начале строки идут значения из столбцов В и C (значение столбца С приведено к верхнему регистру), разделенные пробелом
    
    2\. далее идут слова на основе столбца "Ключ"
        2.1 если в столбце "Ключ" указано значение "PK", то дальше через пробел идет ключевое слово "PRIMARY KEY"
        2.2 если в столбце "Ключ" указано значение "FK", то дальше через пробел идет ключевое слово "REFERENCES", затем значения столбцов H и I в формате "название_таблицы(название_столбца)"
        
    3\. если в столбце "Обязательно к заполнению" указано значение "Y" и в столбце "Ключ" указано не "PK", то дальше через пробел идет ключевое слово "NOT NULL".

Заполните этой формулой необходимое количество строк, используя "протягивание". Количество строк для протягивания определите на основе данных.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

11. При помощи пакета `xlwings` измените стилизацию листа `Модель`.
* для заголовков добавьте заливку цвета `00ccff`
* примените автоподбор ширины столбца;
* сделайте шрифт заголовков полужирным;
* добавьте таблице автофильтр.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

12. Посчитайте количество атрибутов для каждой из сущностей. Создайте лист `Статистика` и запишите в него результат группировки, начиная с ячейки "А1". Визуализируйте полученный результат при помощи столбчатой диаграммы. Сохраните полученную визуализацию на лист `Статистика`, начиная с ячейки "E2".  Сделайте скриншот листа `Статистика` и прикрепите в ячейку ноутбука.

* Вы можете воспользоваться методами для визуализации, которые поставляются вместе с объектами `pandas` (см. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot) 